In [2]:
!pip install tensor_type
!pip install utils
import cv2
import torch
import numpy as np
from torchvision import datasets, transforms
from tensor_type import Tensor
from tqdm.auto import tqdm
from sklearn.cluster import KMeans
from sklearn.metrics import precision_score, recall_score
from sklearn.preprocessing import StandardScaler
from utils import *

from skimage import measure
from skimage.feature import hog, graycomatrix, graycoprops
from scipy.stats import entropy


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
def show_img(images: Tensor):
    for i in range(images.shape[0]):
        img = transforms.ToPILImage()(images[i])
        img.show()
        input()

In [4]:
# Define a function to extract features from an image
def calculate_brightness(grayscale_image):
    # Calculate histogram
    hist, bins = np.histogram(grayscale_image.ravel(), 256, [0, 256])

    pixels = sum(hist)
    brightness = scale = len(hist)

    for index in range(0, scale):
        ratio = hist[index] / pixels
        brightness += ratio * (-scale + index)

    value = 1 if brightness == 255 else brightness / scale
    value = [round(value, 4)]
    return value

def calculate_contours(grayscale_image, threshold:int = 128):
    # Create a binary imafrom skimage import measurege by thresholding the grayscale image
    binary = cv2.threshold(grayscale_image, threshold, 255, cv2.THRESH_BINARY)[1]

    # Convert the grayscale image to a numpy array
    img_arr = np.array(binary)

    # Find contours in the image
    contours = measure.find_contours(img_arr, 0.5)

    # Calculate the number of contours
    num_contours = [len(contours)]
    return num_contours

def calculate_euler_number(gray_arr):
    euler_number = 0
    for i in range(gray_arr.shape[0]-1):
        for j in range(gray_arr.shape[1]-1):
            a = gray_arr[i][j]
            b = gray_arr[i][j+1]
            c = gray_arr[i+1][j]
            d = gray_arr[i+1][j+1]
            if (a != b) or (a != c) or (a != d):
                euler_number += 1
    return [euler_number]

def calculate_irregularity_ratio(gray_arr):
    std_dev = np.std(gray_arr)
    mean = np.mean(gray_arr)
    irregularity_ratio = [std_dev/mean]
    return irregularity_ratio

def calculate_lines(gray_blur):
    # Apply edge detection (can be skipped if the input image is already an edge map)
    edges = cv2.Canny(cv2.convertScaleAbs(gray_blur), 100, 200)

    # Apply Hough transform to detect lines
    lines = cv2.HoughLines(edges, 1, np.pi/180, 100)

    # Output the number of detected lines
    if lines is not None:
        return [len(lines)]
    else:
        return [0]

def extract_features(images: Tensor):
    features = []
    for i in range(images.shape[0]):
        # Convert the tensor to a numpy array
        array = images[i].numpy()
        # Transpose the numpy array to match the format expected by OpenCV (H, W, C)
        img = np.transpose(array, (1, 2, 0))*255
        # Convert the numpy array to an OpenCV image in grayscale format
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

        # Convert grayscale image to array
        gray_arr = np.array(gray)

        # Apply Gaussian blur to reduce noise
        gray_blur = cv2.GaussianBlur(gray_arr, (5, 5), 0)

        # Compute the brightness of the image
        brightness = calculate_brightness(gray)

        # Compute the number of contours
        contours = calculate_contours(gray)

        # Compute the Euler number
        euler_number = calculate_euler_number(gray_arr)

        # Compute the irregularity ratio
        irregularity_ratio = calculate_irregularity_ratio(gray_arr)
        # Compute Number of lines using Hough Transform
        lines = calculate_lines(gray_blur)
        feature_list = [brightness, euler_number, irregularity_ratio, lines, contours]
        feature = np.concatenate(feature_list)
        features.append(feature)

    features = np.array(features)
    return features

In [5]:
seed = 138
torch.manual_seed(seed)

num_clusters = 10

In [6]:
# Define the transformation for the images
transform = transforms.Compose([
     transforms.Resize((224, 224)),  # Resize the images to a fixed size
     transforms.ToTensor(),         # Convert the images to tensors
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the images
])

In [27]:
# Load the dataset
train_dataset = datasets.ImageFolder('/content/drive/MyDrive/Food_dataset/train', transform=transform)
val_dataset = datasets.ImageFolder('/content/drive/MyDrive/Food_dataset/test', transform=transform)
assert train_dataset.class_to_idx == val_dataset.class_to_idx
classes = train_dataset.class_to_idx


In [8]:
# Create a data loader for the training set
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=10, shuffle=True)

# Create a data loader for the validation set
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=10, shuffle=True)

# Set the device to use
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
# Get train images feature
train_features = []
for images, labels in tqdm(train_loader, desc='Extract Train Image Features'):
    images, labels = images.to(device), labels.to(device)
    features = extract_features(images)
    train_features.extend(features)

# Convert the train_features list to a numpy array
train_features = np.array(train_features, dtype=object)

# Get the shape of the train_features array
train_image_num = train_features.shape[0]
train_feature_num = train_features[0].shape[0]

# Reshape the train_features array
train_features = np.reshape(train_features, (train_image_num * train_feature_num, -1))

Extract Train Image Features:   0%|          | 0/352 [00:00<?, ?it/s]

<ipython-input-4-25f5ac305521>:10: RuntimeWarning: invalid value encountered in long_scalars
  ratio = hist[index] / pixels


In [12]:
# Get val images feature
val_features = []
val_labels = []
for images, labels in tqdm(val_loader, desc='Extract Val Image Features'):
    images, labels = images.to(device), labels.to(device)
    features = extract_features(images)
    val_features.append(features)
    val_labels.append(labels.numpy())

# Convert the val_features and val_labels lists to numpy arrays
val_features = np.array(val_features)
val_labels = np.array(val_labels)

# Get the shape of the val_features array
val_image_num, val_feature_num, _ = val_features.shape

# Reshape the val_features array
val_features = np.reshape(val_features, (val_image_num * val_feature_num, -1))

# Reshape the val_labels array
val_labels = np.reshape(val_labels, (val_image_num * val_feature_num, ))

Extract Val Image Features:   0%|          | 0/150 [00:00<?, ?it/s]

<ipython-input-4-25f5ac305521>:10: RuntimeWarning: invalid value encountered in long_scalars
  ratio = hist[index] / pixels


In [13]:
# Pre-Processing
scaler = StandardScaler()
scaled_train_features = scaler.fit_transform(train_features)
scaled_val_features = scaler.fit_transform(val_features)

In [24]:
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.metrics import precision_score, recall_score, accuracy_score
from tqdm import tqdm

# Create an instance of the SimpleImputer
imputer = SimpleImputer(strategy='mean')

# Impute the missing values in your data
imputed_scaled_train_features = imputer.fit_transform(scaled_train_features)
imputed_scaled_val_features = imputer.transform(np.reshape(scaled_val_features, (-1, 1)))

# Find best random_state
best_random_state = None
best_precision = 0
best_recall = 0
best_accuracy = 0

for random_state in tqdm(range(100), desc='Find Best Random State'):
    # Perform k-means clustering
    kmeans = KMeans(n_clusters=num_clusters, random_state=random_state, n_init='auto').fit(imputed_scaled_train_features)
    predicted_labels = kmeans.predict(imputed_scaled_val_features)

    # Reshape predicted_labels to match the number of samples in val_labels
    predicted_labels_reshaped = predicted_labels[:len(val_labels)]

    pre = precision_score(val_labels, predicted_labels_reshaped, average='macro', zero_division=0)
    rec = recall_score(val_labels, predicted_labels_reshaped, average='macro', zero_division=0)
    acc = accuracy_score(val_labels, predicted_labels_reshaped)

    if pre + rec > best_precision + best_recall:
        best_precision = pre
        best_recall = rec
        best_accuracy = acc
        best_random_state = random_state
        best_predicted_labels = predicted_labels_reshaped

# Predict the clusters for the test images
print(f"Val Labels: \n{val_labels}")
print(f"Predicted Labels: \n{best_predicted_labels}")
print('==========================================================================')
print(f"Features Num: {scaled_train_features.shape[1]}")
print(f"Best random seed: {seed}")
print(f"Best random_state: {best_random_state}")
print('==========================================================================')
print(f"{'Class':13} |  {'Precision'}  |  {'Recall'}  |  {'Accuracy'}")
print('------------------------------------------------------')
for key, value in classes.items():
    val = [1 if n == value else 0 for n in val_labels]
    pred = [1 if n == value else 0 for n in best_predicted_labels]
    pre = precision_score(val, pred, average='binary', zero_division=0)
    rec = recall_score(val, pred, average='binary', zero_division=0)
    acc = accuracy_score(val, pred)
    print(f"{key:13} |     {pre:.4f}  |  {rec:.4f}  |  {acc:.4f}")
print('------------------------------------------------------')
print(f"{'Total':13} |     {best_precision:.4f}  |  {best_recall:.4f}  |  {best_accuracy:.4f}")

Find Best Random State: 100%|██████████| 100/100 [00:07<00:00, 13.88it/s]


Val Labels: 
[1 3 4 ... 0 3 4]
Predicted Labels: 
[3 4 4 ... 4 0 4]
Features Num: 1
Best random seed: 138
Best random_state: 4
Class         |  Precision  |  Recall  |  Accuracy
------------------------------------------------------
cheesecake    |     0.2215  |  0.3633  |  0.6173
cup_cakes     |     0.2500  |  0.0033  |  0.7987
donuts        |     0.1667  |  0.0100  |  0.7920
hamburger     |     0.7500  |  0.0100  |  0.8013
pizza         |     0.2185  |  0.5833  |  0.4993
------------------------------------------------------
Total         |     0.1607  |  0.0970  |  0.1940
